### Testowanie uczenia się DFA z pomocą systemu przepisywania więzów

In [24]:
import copy, sys
import random 
import cProfile

sys.path.append("../")
sys.path.append("../inferring")
sys.path.append("../utils")
sys.path.append("../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.oracles.SRS as SRS

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.oracles.SRS import SRS 

from inferring.InferringDFA import InferringDFA

In [25]:
def run_learning_process(test, oracle=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(test.dfa)
    learn_dfa = InferringDFA(_dfa, 
                                oracle, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: liczba - {len(cnt_ex)}, długości - {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa), cnt, cnt_ex

In [26]:
class test_example():
    def __init__(self, dfa, descrip=''):
        self.dfa = copy.deepcopy(dfa)
        self.descrip = descrip

In [4]:
test_examples = []

# 0 ------------------------------------------------------------------------------------------------------------------------------------------------- 
dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1])),
                        dfa2=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1])))
test = test_example(dfa=dfa_, descrip="plot, dfa1 - słowa zawierające literke 'a', dfa2 - słowa zawierające literke 'a'")
test_examples.append(test)
# result, without SRS at all: cnt = [69, 1]

# 1 -------------------------------------------------------------------------------------------------------------------------------------------------
dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):2, (2, 'b'):2}, F= set([2])), 
                        dfa2=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):2, (2, 'b'):2}, F= set([2])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - wzorzec - 'bb', dfa2 - wzorzec 'aa'")
test_examples.append(test)
# result, without SRS at all: cnt = [289, 3]

# 2 -------------------------------------------------------------------------------------------------------------------------------------------------
dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F= set([3])),
                        dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F= set([3])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - wzorzec 'bbb', dfa2 - wzorzec 'aaa'")
test_examples.append(test)
# result, without SRS at all: cnt = [800, 3]

# 3 -------------------------------------------------------------------------------------------------------------------------------------------------
dfa_ = DFA()
dfa_.create_convolution(dfa1 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                        dfa2 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):0, (0, 'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])))
test = test_example(dfa = dfa_, descrip = "splot, dfa1 - wzorzec 'aba', dfa2 - wzorzec 'bbb'")
test_examples.append(test)
# result, without SRS at all: cnt = [1388, 7]

# 4 -------------------------------------------------------------------------------------------------------------------------------------------------
dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                        dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - słowa zawierające i literę 'a' i literę 'b', dfa2 - słowa zawierające i literę 'a' i literę 'b'")
test_examples.append(test)
# result, without SRS at all: cnt = [751, 10]

# 5 -------------------------------------------------------------------------------------------------------------------------------------------------
dfa_ = DFA(Q=3, input_signs=['0', '1', '2', '3', '4', '5', '6', '7'], δ={(0,'0'):0, (0,'1'):2, (0,'2'):2, (0,'3'):0, (0,'4'):2, (0,'5'):0, (0,'6'):1, (0,'7'):2, (1,'0'):2, (1,'1'):0, (1,'2'):1, (1,'3'):2, (1,'4'):1, (1,'5'):2, (1,'6'):2, (1,'7'):1, (2,'0'):2, (2,'1'):2, (2,'2'):2, (2,'3'):2, (2,'4'):2, (2,'5'):2, (2,'6'):2, (2,'7'):2}, F=set([0]), type_=DFA.BITWISE_ADDITION)
test = test_example(dfa=dfa_, descrip="zakodowane dodawania w systemie dwójkowym")
test_examples.append(test)
# result, without SRS at all: cnt = [201, 1]

# 6 -------------------------------------------------------------------------------------------------------------------------------------------------
dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])),
                        dfa2=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - 2 wzorce 'aa' lub 'bb', dfa2 - 2 wzorce 'aa' lub 'bb'" )
test_examples.append(test)
# result, without SRS at all: cnt = [1437, 7]

# 7 -------------------------------------------------------------------------------------------------------------------------------------------------
dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8])),
                        dfa2=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8])))
test = test_example(dfa=dfa_, descrip="splot, dfa1 - 2 wzorce 'aa' i 'bb', dfa2 - 2 wzorce 'aa' i 'bb'")  
test_examples.append(test)
# result, without SRS at all: cnt = [20522, 28]

# 8 -------------------------------------------------------------------------------------------------------------------------------------------------
dfa_ = DFA()
pdfa1_, pdfa2_ = DFA(), DFA()
pdfa1_.create_pattern_dfa(input_signs=['a', 'b'], patterns=['aa', 'bb'], _type="AND")
pdfa2_.create_pattern_dfa(input_signs=['a', 'b', 'c'], patterns=['aa', 'bb', 'c'], _type="AND")

dfa_.create_convolution(dfa1=pdfa1_, dfa2=pdfa2_)

test = test_example(dfa=dfa_, descrip="splot, dfa1 - 2 wzorce: 'aa', 'bb', dfa2 - 3 wzorce: 'aa', 'bb', 'c'")
test_examples.append(test)
# result, without SRS at all: cnt = [121709, 37]

In [ ]:
test = test_examples[5] 
print(test.descrip)
d, _, _ = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False, debug=True)
d, _, _ = run_learning_process(copy.deepcopy(test), oracle=SRS(), check_consistency=False, debug=True)
d, _, _ = run_learning_process(copy.deepcopy(test), oracle=SRS(), check_consistency=True, debug=True)

### Przykład dziwny

In [131]:
patterns1 = ['aaa','bba','cca','ddd']
patterns2 = ['aaa','bbb','ccc','ddd']

pdfa_ = DFA()
pdfa_.create_pattern_dfa(input_signs=['a', 'b', 'c', 'd'], patterns=patterns1, _type="OR")
test_1 = test_example(dfa=pdfa_)

pdfa_ = DFA()
pdfa_.create_pattern_dfa(input_signs=['a', 'b', 'c', 'd'], patterns=patterns2, _type="OR")

test_2 = test_example(dfa=pdfa_)

d1 = run_learning_process(test=test_1)
d2 = run_learning_process(test=test_2)

# kontrprzykłady: [3, 3, 3, 3], cnt = [289, 5]
# kontrprzykłady: [3, 3, 3, 3], cnt = [289, 5]

kontrprzykłady: liczba - 4, długości - [3, 3, 3, 3], cnt = [289, 5]
kontrprzykłady: liczba - 4, długości - [3, 3, 3, 3], cnt = [289, 5]


In [132]:
conv_dfa = DFA() 
conv_dfa.create_convolution(dfa1=d1, dfa2=d2)
d = run_learning_process(test=test_example(dfa=conv_dfa), oracle=SRS(), check_consistency=True, debug=False)
d = run_learning_process(test=test_example(dfa=conv_dfa), oracle=None, check_consistency=False, debug=False)

kontrprzykłady: liczba - 39, długości - [6, 6, 6, 6, 6, 6, 6, 12, 5, 8, 8, 6, 5, 8, 8, 6, 5, 8, 8, 6, 9, 12, 8, 8, 6, 6, 6, 6, 6, 14, 6, 6, 6, 6, 6, 6, 10, 6, 6], cnt = [24258, 4]
kontrprzykłady: liczba - 68, długości - [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], cnt = [69499, 69]


### Duże automaty - ropoznające kilkadziesiąt wzorców (docelowo)

In [12]:
def random_patterns(input_signs, n=5, max_length=10):
    patterns = []
    for _ in range(n):
        p = ''.join(random.choices(population=input_signs,k=random.randint(1, max_length)))
        while p in set(patterns):
            p = ''.join(random.choices(population=input_signs,k=random.randint(1, max_length)))
        patterns.append(p)
    return patterns

random_patterns(input_signs=['a', 'b', 'c', 'd'], max_length=8)

['dcadcd', 'cca', 'ac', 'dacdddaa', 'cdbd']

In [15]:
random.seed(101)

input_signs = ['a', 'b', 'c', 'd']
number_of_patterns = 8
max_pattern_length = 5 

patterns1, patterns2 = random_patterns(input_signs, number_of_patterns, max_pattern_length), random_patterns(input_signs, number_of_patterns, max_pattern_length)
print(f"patterns1 = {patterns1}")
print(f"patterns2 = {patterns2}")

dfa1 = DFA() 
dfa2 = DFA()

dfa1.create_pattern_dfa(input_signs=input_signs, patterns=patterns1, _type=DFA.OR_TYPE_PATTERN_DFA)
print(f"|Q1| = {dfa1.Q}")

dfa2.create_pattern_dfa(input_signs=input_signs, patterns=patterns2, _type=DFA.OR_TYPE_PATTERN_DFA)
print(f"|Q2| = {dfa2.Q}")

d1 = run_learning_process(test_example(dfa=copy.deepcopy(dfa1)))
d2 = run_learning_process(test_example(dfa=copy.deepcopy(dfa2)))

print(f"after learning |Q1| = {d1.Q}")
s1 = ''.join(patterns1)
ans1 = d1.route(s1)

s2 = ''.join(patterns2)
print(f"after learning |Q2| = {d2.Q}")
ans2 = d2.route(s2)

print(f"{ans1}, {ans2}")

patterns1 = ['ddcba', 'aabca', 'aaad', 'ada', 'baba', 'ccdbb', 'acbd', 'cb']
patterns2 = ['b', 'aca', 'aac', 'dca', 'bdcca', 'a', 'bddda', 'dddaa']
|Q1| = 324000
|Q2| = 55296
kontrprzykłady: liczba - 5, długości - [3, 4, 4, 5, 5], cnt = [530, 6]
kontrprzykłady: liczba - 0, długości - [], cnt = [37, 1]
after learning |Q1| = 14
after learning |Q2| = 2
('ddcbaaabcaccdbbaaadbabaacbdadacb', 1), ('bdccabdddadddaaacaaacdcaba', 1)


In [16]:
d2.print_transitions()

(0,a) --> 1
(0,b) --> 1
(0,c) --> 0
(0,d) --> 0
(1,a) --> 1
(1,b) --> 1
(1,c) --> 1
(1,d) --> 1
stany akceptujące - {1}


In [17]:
dfa_ = DFA() 
dfa_.create_convolution(dfa1=copy.deepcopy(d1), dfa2=copy.deepcopy(d2))

dfa_.route(s1+s2.upper())
print(f"|Q_conv(d1,d2)| = {dfa_.Q}")

test = test_example(dfa=dfa_, descrip=f"splot, dfa1 - wzorce: {patterns1}, dfa2 - wzorce: {patterns2}")
print(test.descrip)

|Q_conv(d1,d2)| = 28
splot, dfa1 - wzorce: ['ddcba', 'aabca', 'ccdbb', 'aaad', 'baba', 'acbd', 'ada', 'cb'], dfa2 - wzorce: ['bdcca', 'bddda', 'dddaa', 'aca', 'aac', 'dca', 'b', 'a']


In [ ]:
# cProfile.run('run_learning_process(test, oracle=None, check_consistency=False, debug=False)', sort='time')
run_learning_process(test, oracle=None, check_consistency=False, debug=False)

In [ ]:
# cProfile.run('run_learning_process(test, oracle=SRS(), check_consistency=True, debug=False)', sort='time')
run_learning_process(test, oracle=SRS(), check_consistency=True, debug=False)

### Uczenie się losowych automatów

In [6]:
class Record:
    def __init__(self, d1, d2, conv, cnt_normal, cnt_with_advice):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice
    def print_record(self):
        print(f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, cnt_normal = {self.cnt_normal} vs cnt_with_advice = {self.cnt_with_advice}") 

In [ ]:
seeds = [101, 73, 29, 37, 87, 97, 113, 47, 17, 227]

input_signs = ['a', 'b', 'c', 'd']
max_number_of_states = 60
number_of_itreations = len(seeds)

results = []

i = 0 
while i < number_of_itreations: 
    random.seed(seeds[i])
    i += 1

    dfa1 = DFA()
    dfa2 = DFA()
    dfa1.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    dfa2.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    print(f"(initially) d1_size = {dfa1.Q}, d2_size = {dfa2.Q}")
    
    d1, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa1)))   
    d2, _, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa2)))

    print(f"(minimal) d1_size = {d1.Q}, d2_size = {d2.Q}")
    conv_dfa = DFA()
    conv_dfa.create_convolution(dfa1=d1, dfa2=d2) 
    print(f"(initially) conv_dfa_size = {conv_dfa.Q}")
    
    conv_with_advice, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=conv_dfa), oracle=SRS(), check_consistency=True) 
    conv_with_advice.type = DFA.CONV_DFA
    print(f"(minimal) conv_dfa_size = {conv_with_advice.Q}")
    conv_normal, cnt_normal, cnt_ex_normal = run_learning_process(test=test_example(dfa=conv_with_advice), oracle=None, check_consistency=False)

    results.append(Record(d1=d1.Q, d2=d2.Q, conv=conv_with_advice.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, len(cnt_ex_with_advice))))

In [ ]:
for r in results:
    r.print_record()

### Uczenie się automatów ze słowem synchronizującym

In [40]:
class RecordSYNCH:
    def __init__(self, d, reset_word, cnt_normal, cnt_with_advice):
        self.d = d
        self.reset_word = reset_word
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice
    def print_record(self):
        print(f"|d1| = {self.d}, |reset_word| = {len(self.reset_word)}, cnt_normal = {self.cnt_normal} vs cnt_with_advice = {self.cnt_with_advice}") 

In [ ]:
seeds = [101, 73, 29, 37, 87, 97, 113, 47, 17, 227]

input_signs = ['a', 'b', 'c']
max_number_of_states = 400 # 25
number_of_itreations = len(seeds)

results = []

i = 0 
while i < number_of_itreations: 
    random.seed(seeds[i])
    i += 1

    reset_word = DFA.NOT_RESETING_WORD
    idx = 0 
    while True:
        dfa_ = DFA()
        dfa_.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
        print(f"idx = {idx}: d1_size (initially) = {dfa_.Q}")
        d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_)))   
        print(f"(minimal) d1_size = {d.Q}")

        _, reset_word = d.check_synchronicity()
        print(f"reset_word = {reset_word}")

        if reset_word!=DFA.NOT_RESETING_WORD:
            d.type = DFA.SYNCHRONICITY
            d.reset_word = reset_word
            break 
        idx += 1 
    
    d1_with_advice, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d), oracle=SRS(), check_consistency=True) 
    d1_with_advice.type = DFA.SYNCHRONICITY

    results.append(RecordSYNCH(d=d.Q, reset_word=reset_word, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, len(cnt_ex_with_advice))))

In [119]:
for r in results:
    r.print_record()

|d1| = 317, |reset_word| = 84, cnt_normal = [60007, 72] vs cnt_with_advice = ([180301, 33], 35)
|d1| = 257, |reset_word| = 70, cnt_normal = [22402, 30] vs cnt_with_advice = ([39829, 7], 7)
|d1| = 327, |reset_word| = 82, cnt_normal = [25872, 28] vs cnt_with_advice = ([109053, 11], 12)
|d1| = 359, |reset_word| = 104, cnt_normal = [17615, 16] vs cnt_with_advice = ([77978, 4], 4)
|d1| = 227, |reset_word| = 69, cnt_normal = [8872, 14] vs cnt_with_advice = ([34739, 6], 6)
|d1| = 236, |reset_word| = 51, cnt_normal = [11588, 14] vs cnt_with_advice = ([25219, 2], 2)
|d1| = 197, |reset_word| = 73, cnt_normal = [11652, 22] vs cnt_with_advice = ([30590, 5], 5)
|d1| = 285, |reset_word| = 75, cnt_normal = [12847, 12] vs cnt_with_advice = ([44730, 3], 3)
|d1| = 314, |reset_word| = 82, cnt_normal = [14781, 18] vs cnt_with_advice = ([54928, 5], 5)
|d1| = 290, |reset_word| = 91, cnt_normal = [12491, 14] vs cnt_with_advice = ([54295, 3], 3)


### Uczenie się automatów typu **marked words**

In [27]:
dfa_ = DFA()
dfa_.create_pattern_dfa(input_signs=['a', 'b', 'c'], patterns=['abcba'])
dfa_.route(w="abbabcbaa")
dfa_lm = dfa_.create_marked_words_atomaton()
w = dfa_lm.find_accepting_word(start_state=0)
print(f"w = {w}")
ans = dfa_lm.route("αabbaβ")
print(f"ans = {ans}")

w = (True, 'αabcbaβ')
ans = ('αabbaβ', 0)


In [28]:
import sys
# sys.path.insert(0, "..")

from utils.oracles.SRSmark import SRSmark

srs = SRSmark(alphabet=['a', 'b', 'c'])
nw = srs.get_normal_form(w="αabbaβ")
nw = srs.get_normal_form(w="αβ")
nw = srs.get_normal_form(w="aabbcαβ")
nw = srs.get_normal_form(w="αβaabbc")
nw = srs.get_normal_form(w="bbcaαβaabbc")
nw = srs.get_normal_form(w="αbbcaαβaaαaabbcbbcβ")

print(f"nfrorm of w = {nw}")

nfrorm of w = αα


In [29]:
class RecordMARKED:
    def __init__(self, d, d_marked_cnt_normal, d_marked_cnt_with_advice):
        self.d = d
        self.d_marked_cnt_normal = d_marked_cnt_normal
        self.d_marked_cnt_with_advice = d_marked_cnt_with_advice
    def print_record(self):
        print(f"|d1| = {self.d}, |d_marked_cnt_normal| = {self.d_marked_cnt_normal} vs |d_marked_cnt_with_advice| = {self.d_marked_cnt_with_advice}") 

In [58]:
seeds = [101] #, 73, 29, 37, 87, 97, 113, 47, 17, 227]

input_signs = ['a', 'b', 'c', 'd',] # 'e', 'f']
max_number_of_states = 20 # 25
number_of_itreations = len(seeds)

results = []

i = 0 
while i < number_of_itreations:  
    random.seed(seeds[i])
    i += 1

    dfa_ = DFA()
    dfa_.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    print(f"WULOSOWANY dfa! dfa_size (initially) = {dfa_.Q}\n\n")

    d, cnt_normal, _ = run_learning_process(test_example(dfa=copy.deepcopy(dfa_)), oracle=None, check_consistency=False, debug=True)   
    print(f"NAUCZONY dfa! (minimal) d1_size = {d.Q}\n\n")

    d_marked = d.create_marked_words_atomaton()
    print(f"stworzyłem MARKED dfa!\n\n")

    _, cnt_normal, _ = run_learning_process(test=test_example(dfa=d_marked))
    _, cnt_with_advice, cnt_ex_with_advice = run_learning_process(test=test_example(dfa=d_marked), oracle=SRS(), check_consistency=True, debug=True) 
    print(f"NAUCZONY MARKED dfa! z więzami")

    results.append(RecordMARKED(d=d.Q, d_marked_cnt_normal=cnt_normal, d_marked_cnt_with_advice=(cnt_with_advice, len(cnt_ex_with_advice))))

WULOSOWANY dfa! dfa_size (initially) = 19


S = 1, rozmiar E = 5
zamkniętość sprawdzona - S = 15, rozmiar E = 5
hipoteza: 15
(0,a) --> 1
(0,b) --> 2
(0,c) --> 3
(0,d) --> 4
(1,a) --> 3
(1,b) --> 5
(1,c) --> 6
(1,d) --> 1
(2,a) --> 7
(2,b) --> 8
(2,c) --> 9
(2,d) --> 7
(3,a) --> 4
(3,b) --> 1
(3,c) --> 2
(3,d) --> 0
(4,a) --> 9
(4,b) --> 6
(4,c) --> 9
(4,d) --> 10
(5,a) --> 3
(5,b) --> 0
(5,c) --> 3
(5,d) --> 3
(6,a) --> 0
(6,b) --> 10
(6,c) --> 9
(6,d) --> 11
(7,a) --> 9
(7,b) --> 12
(7,c) --> 9
(7,d) --> 12
(8,a) --> 5
(8,b) --> 5
(8,c) --> 3
(8,d) --> 6
(9,a) --> 3
(9,b) --> 1
(9,c) --> 12
(9,d) --> 13
(10,a) --> 1
(10,b) --> 0
(10,c) --> 13
(10,d) --> 10
(11,a) --> 9
(11,b) --> 14
(11,c) --> 4
(11,d) --> 2
(12,a) --> 14
(12,b) --> 11
(12,c) --> 5
(12,d) --> 3
(13,a) --> 14
(13,b) --> 5
(13,c) --> 6
(13,d) --> 0
(14,a) --> 8
(14,b) --> 12
(14,c) --> 1
(14,d) --> 13
stany akceptujące - {1, 2, 4, 6, 7, 12, 14}
kontrprzyklad (z zapytania o równoważność)= aaac



S = 15, rozmiar E = 6
za

In [57]:
for r in results:
    r.print_record()

|d1| = 19, |d_marked_cnt_normal| = [425, 3] vs |d_marked_cnt_with_advice| = ([824, 7], 7)


In [ ]:
|d1| = 1, |d_marked_cnt_normal| = [13, 1] vs |d_marked_cnt_with_advice| = ([23, 1], 0) alfabet - 3
|d1| = 4, |d_marked_cnt_normal| = [40, 1] vs |d_marked_cnt_with_advice| = ([126, 4], 4) alfabet - 3 
|d1| = 14, |d_marked_cnt_normal| = [217, 4] vs |d_marked_cnt_with_advice| = ([442, 7], 7) alfabet - 3 
|d1| = 19, |d_marked_cnt_normal| = [425, 3] vs |d_marked_cnt_with_advice| = ([824, 7], 7) alfabet - 4 

### Uczenie się automatów z częściową specyfikacją

### Uczenie się splotu DFA - **debugowanie** 

In [ ]:
dfa1 = DFA()
dfa2 = DFA()

dfa1.create_random_dfa(Q=1, input_signs=['a', 'b', 'c'])
dfa2.create_pattern_dfa(input_signs=['a', 'b', 'c'], patterns=['aa'])

dfa1.print_transitions()
dfa2.print_transitions()

conv_dfa =DFA()
conv_dfa.create_convolution(dfa1=dfa1, dfa2=dfa2)

conv_dfa.print_transitions()
d, x, y = run_learning_process(test=test_example(dfa=conv_dfa))
d.print_transitions()

In [60]:
import BitVector
bv = BitVector.BitVector(bitlist =[1, 1, 0, 1])
print(f"bv = {bv}, int(bv) = {int(bv)}")

bv_ = BitVector.BitVector(bitlist = [1])
bv = bv + bv_

print(f"bv = {bv}, int(bv) = {int(bv)}")

bv = 1101, int(bv) = 13
bv = 11011, int(bv) = 27


In [9]:
from itertools import product 

q1, q2 = 5, 4
xs = product(range(q1), repeat=2) 

for i, x in enumerate(xs):
    print(i, x)

print(3**2)


0 (0, 0)
1 (0, 1)
2 (0, 2)
3 (0, 3)
4 (0, 4)
5 (1, 0)
6 (1, 1)
7 (1, 2)
8 (1, 3)
9 (1, 4)
10 (2, 0)
11 (2, 1)
12 (2, 2)
13 (2, 3)
14 (2, 4)
15 (3, 0)
16 (3, 1)
17 (3, 2)
18 (3, 3)
19 (3, 4)
20 (4, 0)
21 (4, 1)
22 (4, 2)
23 (4, 3)
24 (4, 4)
9


<!-- ### Uczenie się splotów - **debugowanie**  -->